In [4]:
% matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas
import torch, torch.utils.data, torchvision
import PIL
import os.path
import time
import skimage, skimage.io
import time
import copy
from my_utils import *

In [5]:
MEAN = [0.485, 0.456, 0.406] # expected by pretrained resnet18
STD = [0.229, 0.224, 0.225] # expected by pretrained resnet18

# Full model

In [6]:
# load the data and add a column for occurrences
df = pandas.read_csv('./data/train.csv')
grouped = df.groupby('Id')
df['occurrences'] =  grouped.Id.transform('count')

In [8]:
# define transformations with data augmentation.
# These are the same transformations that were used for the toy_model
transforms_augm = torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(degrees=20.0), # Data augmentation
    torchvision.transforms.RandomGrayscale(), # Data augmentation
    # torchvision.transforms.Resize((224,int(224.0*16.0/9.0))),
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(), # Expected by pretrained neural network
    torchvision.transforms.Normalize(MEAN, STD) # Expected by pretrained neural network
    ])

In [9]:
# Load the full dataset. Using a random subset as the validation set is difficult because
# many of the categories appear only a single time. I could try to use cross-validation instead,
# but for the moment I am just going to ignore the validation set.
full_data = WhaleDataset(df,'./data/',transform=transforms_augm)
full_dataloader = {'train': torch.utils.data.DataLoader(full_data,\
                                                   batch_size=4,\
                                                   num_workers=4,\
                                                   shuffle=True,\
                                                   sampler=None)}

In [10]:
# load the toy_model
toy_model = torch.load('./toy_model_all_layers_trained.pt')

In [11]:
# Create the full model from the toy nodel by replacing the last layer
full_model = copy.deepcopy(toy_model)

# freeze layers
for params in full_model.parameters():
    params.requires_grad == False

# Create new final layer (unfrozen by default)
num_in = full_model.fc.in_features
num_out = len(full_dataloader['train'].dataset.categories)
full_model.fc = torch.nn.Linear(num_in,num_out)
full_model.fc

# define the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
# Choose the loss function (criterion) and optimizer.
# I make the same choice as for the toy_model.
criterion = torch.nn.CrossEntropyLoss()
full_optimizer = torch.optim.Adam(toy_model.fc.parameters(),lr=0.001)

In [ ]:
# train the full model
full_model, loss_vals, acc_vals = train_with_restart(
    full_model,full_dataloader,criterion,full_optimizer,device,\
    use_val=False,num_epochs=10)

Epoch 0 / 10


In [5]:
torch.save(full_model,'full_model.pt')

NameError: name 'full_model' is not defined